In [1]:
import sys
import cobra
#If you have CPLEX, uncomment this
#import cplex
import cobrakbase


cobrakbase 0.2.7


In [20]:
data = ""
with open('AA_residues.tsv', 'r') as file:
    data = file.read()
lines = data.split("\n")
residue_data = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        residue_data[array[0]+"_c0"] = {
            "name":array[1],
            "structure":array[2],
            "formula:":array[3],
            "mass":array[4],
            "exact_mass":array[5]
        }

modifications = {}
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("AAResidueDamageProducts",95771)
kbmodel = kbase_api.get_object("AAResidueDamageProducts",95771)
rxnops = {}
for rxn in kbmodel["modelreactions"]:
    rxnops[rxn["id"]] = []
    if "dblinks" in rxn and "PickAxe" in rxn["dblinks"]:
        for op in rxn["dblinks"]["PickAxe"]:
            op = op.replace("enzymatic.","")
            op = op.replace("spontaneous.","")
            rxnops[rxn["id"]].append(op)
for reaction in model.reactions:
    reactant = None
    for metabolite in reaction.metabolites:
        if metabolite.id in residue_data:
            reactant = metabolite
            #print(reactant.id+":"+str(reactant.formula_weight)+":"+residue_data[reactant.id]["mass"]+":"+residue_data[reactant.id]["exact_mass"])   
    if reactant != None:
        closest_metabolite = None
        closest_mass = None
        for metabolite in reaction.metabolites:
            if reaction.metabolites[metabolite] > 0 and metabolite != reactant:
                if closest_mass == None or abs(metabolite.formula_weight - reactant.formula_weight) < closest_mass:
                    closest_mass = abs(metabolite.formula_weight - reactant.formula_weight)
                    closest_metabolite = metabolite
        if reactant.id not in modifications:
            modifications[reactant.id] = {}
        if closest_mass not in modifications[reactant.id]:
            modifications[reactant.id][closest_mass] = {"count":0,"ops":[]}
        modifications[reactant.id][closest_mass]["count"] += 1
        for op in rxnops[reaction.id]:
            if op not in modifications[reactant.id][closest_mass]:
                modifications[reactant.id][closest_mass]["ops"].append(op)

print("Residue\tMassdiff\tCount\tOperators")
for residue in modifications:
    for massdiff in modifications[residue]:
        print(residue+"\t"+str(massdiff)+"\t"+str(modifications[residue][massdiff]["count"])+"\t"+"|".join(modifications[residue][massdiff]["ops"]))

Residue	Massdiff	Count	Operators
ThreonineResidue_c0	192.21294	1	1.1.1_02.rev
ThreonineResidue_c0	1.00794	5	1.4.1_01|2.1.1_02|AmineMethylation|2.1.1_02|2.1.1_02|AmineMethylation|1.1.1_11
ThreonineResidue_c0	30.025979999999997	2	1.4.3_01|1.4.3_01
ThreonineResidue_c0	75.0666	1	3.7.1_02
ThreonineResidue_c0	31.0571	2	1.4.3_01|3.5.1_02
ThreonineResidue_c0	18.01528	5	1.14.13_02|1.14.13_10|1.14.13_02|1.14.13_10|1.14.13_10
ThreonineResidue_c0	226.167461	1	2.7.1_08
ThreonineResidue_c0	254.28232	4	1,2-BenzoquinoneBetaAddition_1,2Benzoquinone|1,4-BenzoquinoneAddition_1,4-Benzoquinone|1,2-BenzoquinoneBetaAddition_1,2Benzoquinone|1,4-BenzoquinoneAddition_1,4-Benzoquinone
ThreonineResidue_c0	97.99518099999999	2	Carbamylation_CarbamoylPhosphate|Carbamylation_CarbamoylPhosphate
ThreonineResidue_c0	17.03052	2	Carbamylation_Urea|Carbamylation_Urea
PhenylalanineResidue_c0	18.01528	10	1.14.15_02|1.14.13_08|1.14.13_10|1.14.13_02|1.14.13_10|1.14.13_08|1.14.13_10|1.14.13_08|1.14.13_10|1.14.13_02
Phenylalanin